In [1]:
import numpy as np
from instrumental import Q_, u
from instrumental import instrument, list_instruments
from instrumental.drivers.daq.ni import Task
import matplotlib.pyplot as plt

In [2]:
paramset = list_instruments()
paramset

C:\Users\Prismo\miniconda3\lib\site-packages\visa.py:13: FutureWarning: The visa module provided by PyVISA is being deprecated. You can replace `import visa` by `import pyvisa as visa` to achieve the same effect.

The reason for the deprecation is the possible conflict with the visa package provided by the https://github.com/visa-sdk/visa-python which can result in hard to debug situations.
  warnings.warn(
C:\Users\Prismo\miniconda3\lib\site-packages\instrumental_lib-0.8.dev0-py3.9.egg\instrumental\drivers\motion\ecc100.py:24: UserWarning: Driver 'instrumental.drivers.motion.ecc100' is out of date and incompatible with the current Instrumental core. Pull requests fixing this are welcome.
  warnings.warn(


[<ParamSet[HPMultimeter] visa_address='GPIB0::20::INSTR'>,
 <ParamSet[UC480_Camera] serial=b'4103485838' model=b'C1285R12M' id=1>,
 <ParamSet[NIDAQ] name='Dev2' serial=20316885 model=b'USB-6259'>,
 <ParamSet[BPC203] serial='71829423'>]

In [3]:
from instrumental.conf import user_conf_dir
user_conf_dir

'C:\\Users\\Prismo\\AppData\\Local\\mabuchilab\\instrumental'

stage.save_instrument('NanoMax_stage')
from instrumental.drivers.motion.BPC203 import BPC203_stage

In [3]:
%cd ..
from experiment_control.srs_microscope.srs_microscope import *

C:\Program Files (x86)\Thorlabs


ModuleNotFoundError: No module named 'experiment_control'

In [ ]:
cam = instrument(paramsets[0]) cam.save_instrument('Thorlabs_camera')

In [ ]:
import sys
import clr
import os

sys.path.append(r"C:\Program Files\Thorlabs\Kinesis")
os.getcwd()
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.FilterFlipperCLI")
# from instrumental.drivers.motion.filter_flipper import Position

In [ ]:
ff      = instrument("Thorlabs_FilterFlipper",reopen_policy="reuse")

In [ ]:
cam = instrument("Thorlabs_camera",reopen_policy="reuse")

ff = instrument(paramsets[2])
ff.save_instrument('Thorlabs_FilterFlipper')

daq = instrument(paramsets[1])
daq.save_instrument('NIDAQ_USB-6259')

daq = instrument('NIDAQ_USB-6259')

In [ ]:
#Configure DAQ input channels
Vi, Vi_str = daq.ai0, 'Dev2/ai0'
#configure DAQ output channels
Vo, Vo_str = daq.ao0, 'Dev2/ao0'

n_samp = 100
fsamp = 1000*u.Hz
test_task = Task(Vi, Vo)
test_task.set_timing(fsamp = fsamp, n_samples=n_samp)


In [ ]:
Vo.write(0*u.V)

In [ ]:
V_data = np.linspace(0,1,n_samp//2)*u.V
V_data = np.concatenate((V_data, V_data[::-1]))
# V_data = np.ones(5)*0*u.V
write_data = {Vo_str: V_data}
read_data = test_task.run(write_data)
test_task.unreserve()
print(read_data)
print(V_data)

# Testing plotting